In [30]:
from instagrapi import Client
from instagrapi.exceptions import LoginRequired
import logging
import time
import sqlite3
import pandas as pd
from time import localtime
# import httperror
from urllib.error import HTTPError
import re

In [23]:

# USERNAME = 'coolpolyester'
# PASSWORD = 'Aaronjk12'

In [91]:
# Should not need to run this ever again. 
# cl = Client()
# cl.login(USERNAME, PASSWORD)
# cl.delay_range = [3, 10]
# cl.dump_settings("session.json")

True

In [4]:
logger = logging.getLogger()


def login_user(cl):
    """
    Attempts to login to Instagram using either the provided session information
    or the provided username and password.
    """
    USERNAME = 'coolpolyester'
    PASSWORD = 'Aaronjk12'
    
    session = cl.load_settings("session.json")

    login_via_session = False
    login_via_pw = False

    if session:
        try:
            cl.set_settings(session)
            cl.login(USERNAME, PASSWORD)

            # check if session is valid
            try:
                cl.get_timeline_feed()
            except LoginRequired:
                logger.info(
                    "Session is invalid, need to login via username and password"
                )

                old_session = cl.get_settings()

                # use the same device uuids across logins
                cl.set_settings({})
                cl.set_uuids(old_session["uuids"])

                cl.login(USERNAME, PASSWORD)
            login_via_session = True
        except Exception as e:
            logger.info("Couldn't login user using session information: %s" % e)

    if not login_via_session:
        try:
            logger.info(
                "Attempting to login via username and password. username: %s" % USERNAME
            )
            if cl.login(USERNAME, PASSWORD):
                login_via_pw = True
        except Exception as e:
            logger.info("Couldn't login user using username and password: %s" % e)

    if not login_via_pw and not login_via_session:
        raise Exception("Couldn't login user with either password or session")

In [99]:
cl = Client()
login_user(cl)
# user_id = cl.user_id_from_username('actuarialclubuw')
# time.sleep(3)

In [91]:
conn = sqlite3.connect('clubs.db')
c = conn.cursor()
# instaUrls = c.execute(r'SELECT club_id, instagramUrl FROM clubs WHERE instagramUrl;').fetchall()

In [100]:
clubs_df = pd.read_sql_query("SELECT club_id, instagramUrl, insta_id FROM clubs;", conn)
# strftime('%Y-%m-%d %H:%M:%S', localtime(1347517370))
# cl.delay_range = [3, 10]

def extract_first_resource(url):
    # This regular expression will find the first resource after .com/ in the URL
    match = re.search(r'\.com/([^/?]+)', url)
    return match.group(1) if match else None


for index, club in clubs_df.iterrows():
    try:
        if club["instagramUrl"] != "" and club["insta_id"] is None:
            url = club["instagramUrl"]
            username = extract_first_resource(url)
            print(f'Requesting {username}\'s user_id')
            user_id = cl.user_id_from_username(username)

            # Put user_id into database
            c.execute('UPDATE clubs SET insta_id = ? WHERE club_id = ?;', (user_id, club['club_id']))
            conn.commit()

            formatted_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
            print(index, f'{club["instagramUrl"]}\'s user_id is {user_id} at {formatted_time}')
            
        elif club["instagramUrl"] != "":
            print(index, club["instagramUrl"] + "'s user_id is already in database")
    except HTTPError as e:
        print(e)
        continue

1 https://www.instagram.com/actuarialclubuw/'s user_id is already in database
2 https://www.instagram.com/alps_uw/'s user_id is already in database
3 https://www.instagram.com/adclubwisc's user_id is already in database
4 https://www.instagram.com/airwisco/'s user_id is already in database
5 https://www.instagram.com/ahana_maps/'s user_id is already in database
6 http://instagram.com/aigauw's user_id is already in database
7 https://www.instagram.com/ahswisconsin/'s user_id is already in database
9 http://www.instagram.com/alphachisigma_uw's user_id is already in database
10 https://www.instagram.com/alphadeltaphiwisco/'s user_id is already in database
11 https://www.instagram.com/aed.uwmadison/'s user_id is already in database
12 https://www.instagram.com/agr_uw/'s user_id is already in database
13 https://www.instagram.com/wisconsinakdphi/'s user_id is already in database
14 https://www.instagram.com/badgerakpsi's user_id is already in database
15 https://www.instagram.com/aomegae_uw

In [90]:
# user_id = cl.user_id_from_username('aspiringnursesassociation')
# user_id

# dump it into sql db
# c.execute('UPDATE clubs SET insta_id = ? WHERE club_id = ?;', ('61487313454', '/organization/ana'))

# Remove instagram url https://www.instagram.com/badgerloop/
conn.close()
conn = sqlite3.connect('clubs.db')	
c = conn.cursor()
c.execute('UPDATE clubs SET instagramUrl = ? WHERE club_id = ?;', ('', '/organization/ceramicsclub'))
conn.commit()
conn.close()

In [81]:
# How many unique insta urls
conn = sqlite3.connect('clubs.db')
c = conn.cursor()
pd.read_sql_query("SELECT COUNT(DISTINCT insta_id) FROM clubs;", conn)

,COUNT(DISTINCT insta_id)
0,321
